In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import joblib

# Load the saved model
loaded_model = joblib.load('/content/drive/MyDrive/model_filename_original_resnet.pkl')

In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import joblib  # Import joblib for model saving
import seaborn as sns  # Import seaborn for visualization
import matplotlib.pyplot as plt  # Import matplotlib for visualization
from sklearn.linear_model import LogisticRegression

# Define batch size
batch_size = 32

# Define the base folder path to your dataset
base_path = '/content/drive/MyDrive/Celeb-DFF'

# Define subfolder paths for real and fake videos
real_path_1 = os.path.join(base_path, 'Celeb-real')
fake_path = os.path.join(base_path, 'Celeb-synthesis')

# Collect file paths for real and fake videos
fake_video_files = [os.path.join(fake_path, file) for file in os.listdir(fake_path) if file.endswith('.mp4')]
real_video_files = [os.path.join(real_path_1, file) for file in os.listdir(real_path_1) if file.endswith('.mp4')]

# Create labels
real_labels = ['echt'] * len(real_video_files)
fake_labels = ['df'] * len(fake_video_files)

# Combine file paths and labels
file_paths = real_video_files + fake_video_files
labels = real_labels + fake_labels

# Split the dataset into training (80%) and validation (20%)
X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# Load the pretrained ResNet-50 model with weights from ImageNet
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Create the final feature extraction model
feature_extraction_model = Model(inputs=base_model.input, outputs=x)

# Function to load and preprocess video frames with tqdm progress bars
def load_and_preprocess_video(file_path, label, desired_num_frames=16):
    cap = cv2.VideoCapture(file_path)
    frames = []

    # Read frames from the video
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for _ in tqdm(range(min(num_frames, desired_num_frames)), desc=f"Processing {os.path.basename(file_path)}", unit=" frame"):
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to match ResNet-50 input size
        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        frame = np.expand_dims(frame, axis=0)  # Add batch dimension

        frames.append(frame)

    cap.release()

    # Check if any frames were read
    if not frames:
        return None, label

    # Pad or truncate frames to the desired number
    if len(frames) < desired_num_frames:
        frames = np.concatenate([frames] * (desired_num_frames // len(frames) + 1), axis=0)
        frames = frames[:desired_num_frames]
    elif len(frames) > desired_num_frames:
        frames = frames[:desired_num_frames]

    frames = np.vstack(frames)  # Stack frames into a single array
    return frames, label

# Use the feature extraction model to extract features from videos
train_features = []
val_features = []

for file_path, label in zip(X_train, y_train):
    frames, _ = load_and_preprocess_video(file_path, label)
    if frames is not None:
        features = feature_extraction_model.predict(frames)
        train_features.append(features)

for file_path, label in zip(X_val, y_val):
    frames, _ = load_and_preprocess_video(file_path, label)
    if frames is not None:
        features = feature_extraction_model.predict(frames)
        val_features.append(features)

# Convert the extracted features to numpy arrays
X_train_features = np.array(train_features).reshape(len(train_features), -1)
X_val_features = np.array(val_features).reshape(len(val_features), -1)

# Define an increased hyperparameter search space for Grid Search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [100, 200, 300, 400, 500, 1000],
    'solver': ['lbfgs', 'liblinear', 'newton-cg'],
    'penalty': ['l1', 'l2', 'elasticnet'],
}

# Initialize logistic regression classifier
classifier = LogisticRegression()

# Initialize and run GridSearchCV
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_features, y_train)

# Save the optimized model to your Google Drive directory
joblib.dump(grid_search.best_estimator_, '/content/drive/MyDrive/grid_search_optimized.pkl')

# Predictions on validation data using the best estimator from grid search
y_val_pred = grid_search.best_estimator_.predict(X_val_features)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_val_pred)

# Calculate precision
precision = precision_score(y_val, y_val_pred, pos_label='df')

# Calculate ROC AUC
y_val_proba = grid_search.best_estimator_.predict_proba(X_val_features)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_proba)

# Calculate recall
recall = recall_score(y_val, y_val_pred, pos_label='df')

# Calculate F1-score
f1 = f1_score(y_val, y_val_pred, pos_label='df')

# Create and visualize a confusion matrix heatmap
cm = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['echt', 'df'], yticklabels=['echt', 'df'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix Heatmap')
plt.show()

# Display metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


In [ ]:
# Increase the number of iterations (max_iter) or scale the data as shown in:
#     https://scikit-learn.org/stable/modules/preprocessing.html
# Please also refer to the documentation for alternative solver options:
#     https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
#   n_iter_i = _check_optimize_result(

# Accuracy: 0.73
# Precision: 0.70
# ROC AUC: 0.70
# Recall: 0.91
# F1-score: 0.79